In [1]:
import datetime

import backtrader

import lstm

Using TensorFlow backend.


In [2]:
# datafeed to provide indexes to frames to be used by a strategy
class FrameFeed(backtrader.feed.DataBase):
    params = (
        ("fromdate", datetime.datetime),
        ("todate", datetime.datetime),
        ("frames", None),
    )

    lines = ("datetime", "frame_index", "close")

    def __init__(self):
        super()

    def start(self):
        super(backtrader.feed.DataBase, self).start()
        # TODO: use assignment expressions
        self.frames = (
            frame for frame in enumerate(self.p.frames)
            if self.p.fromdate <= frame[1][-1][-1][0] <= self.p.todate
        )

    def _load(self):
        try:
            i, frame = next(self.frames)
        except StopIteration:
            return False

        self.lines.datetime[0] = backtrader.utils.date2num(frame[-1][-1][0])
        self.lines.frame_index[0] = i

        return True

In [38]:
# strategy to use by backtesting
class TestStrategy(backtrader.Strategy):
    params = (
        ("model", None),
        ("frames", [[[]]]),
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt, txt))

    def __init__(self):
        self.closings = self.datas[0].close
        self.frame_index = self.datas[1].frame_index
        self.frames = self.params.frames
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        print(f"order size: {order.size}")

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED:')

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log("SELL EXECUTED:")

            print(f"Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}")
            print(self.broker.getvalue())

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        if self.order:
            return

        current_frame = [[self.closings[i]] for i in range(0, -15, -1)]
        previous_frame = [[self.closings[i]] for i in range(-1, -16, -1)]

        current_normalized_frame = lstm.normalize_frame(current_frame)
        previous_normalized_frame = lstm.normalize_frame(previous_frame)
    
        current_prediction = lstm.predict_sequences_multiple(self.p.model, [current_normalized_frame])
        previous_prediction = lstm.predict_sequences_multiple(self.p.model, [previous_normalized_frame])
        
        current_prediction_denorm = lstm.denormalize_dim(current_prediction[0][0], current_frame[0][0])
        # previous_prediction_denorm = lstm.denormalize_dim(previous_prediction[0][0], previous_frame[0][0])
        
        # print(f"current: {current_frame}, prediction: {current_prediction_denorm}")
        
        pred_diff = current_prediction[0][0] - previous_prediction[0][0]
        
        # print(f"{current_prediction} - {previous_prediction} = {diff}")
        
        # note: track position and buy/sell entire stake
#         if pred_diff > 0 and current_prediction_denorm > current_frame[-1][0]:
#             if not self.position:
#                 self.log("BUY")
#                 self.order = self.buy()
#         elif self.position:
#             self.log("CLOSING LONG")
#             self.order = self.close()
            
#         if pred_diff < 0 and current_prediction_denorm < current_frame[-1][0]:
#             if not self.position:
#             self.log("SHORT")
#             self.order = self.sell()
#         elif self.position:
#             self.log("CLOSING SHORT")
#             self.order = self.close()

        if pred_diff > 0 and current_prediction_denorm > current_frame[-1][0]:
            if not self.position:
                self.log("BUY")
                self.order = self.buy()
        elif pred_diff < 0 and current_prediction_denorm < current_frame[-1][0]:
            if not self.position:
                self.log("SHORT")
                self.order = self.sell()
        elif self.position:
            self.log("CLOSING")
            self.order = self.close()
            
                

In [4]:
class Reverser(backtrader.sizers.FixedSize):

    def _getsizing(self, comminfo, cash, data, isbuy):
        position = self.broker.getposition(data)
        return cash // data.open[0] - 1
#         size = self.p.stake * (1 + (position.size != 0))
#         return size
#         if isbuy:
#             return cash // data.open[0] - 1
#         else:
#             return position.size

In [40]:
SYMBOL="MSFT"

# get testing data
times = lstm.get_time_series_daily(SYMBOL, ["1. open"], outputsize="full")
vectors = lstm.times_to_vectors(times, include_time=True)[::-1]

train_vectors, test_vectors = lstm.partition_data(vectors)

train_frames = lstm.get_frames(train_vectors, 15, with_target=True)
test_frames = lstm.get_frames(test_vectors, 15, with_target=False)

train_no_dates = [[[col[1] for col in vector] for vector in frame] for frame in train_frames]
normalized_train = lstm.normalize_frames(train_no_dates)
x_train, y_train = lstm.seperate_xy(normalized_train)

In [41]:
# setup model
model = lstm.setup_lstm_model(x_train, y_train)

compilation time :  0.022873878479003906
Train on 4035 samples, validate on 213 samples
Epoch 1/1
4035/4035 [==============================] - 3s 809us/step - loss: 1.0095 - val_loss: 0.9881


In [42]:
# setup inital testing strategy
cerebro = backtrader.Cerebro()
cerebro.broker.setcash(100000.0)
cerebro.addsizer(Reverser)
cerebro.broker.setcommission(commission=0.001)

# add data and model to strategy
cerebro.addstrategy(TestStrategy, model=model, frames=test_frames)

from_date = datetime.datetime(2018, 3, 1)
to_date = datetime.datetime(2019, 3, 1)

cerebro.adddata(
    backtrader.feeds.YahooFinanceData(
        dataname=SYMBOL,
        fromdate=from_date,
        todate=to_date
    )
)

cerebro.adddata(
    FrameFeed(
        frames=test_frames,
        fromdate=from_date,
        todate=to_date
    )
)

print(f'Beginning Portfolio Value: {cerebro.broker.getvalue()}')

cerebro.run()

print(f'Final Portfolio Value: {cerebro.broker.getvalue()}')

Beginning Portfolio Value: 100000.0
2018-03-01, BUY
order size: 1080.0
2018-03-02, BUY EXECUTED:
Price: 90.06, Cost: 97264.8, Comm: 97.26480000000001
101468.7352
2018-03-02, CLOSING
order size: -1080.0
2018-03-05, SELL EXECUTED:
Price: 90.81, Cost: 97264.8, Comm: 98.07480000000001
100614.6604
2018-03-08, SHORT
order size: -1084.0
2018-03-09, SELL EXECUTED:
Price: 93.71, Cost: -101581.64, Comm: 101.58164000000001
99179.75876000003
2018-03-13, CLOSING
order size: 1084.0
2018-03-14, BUY EXECUTED:
Price: 93.54, Cost: -101581.64, Comm: 101.39736000000002
100595.9614
2018-03-14, SHORT
order size: -1074.0
2018-03-15, SELL EXECUTED:
Price: 91.98, Cost: -98786.52, Comm: 98.78652000000001
99809.81487999999
2018-03-19, CLOSING
order size: 1074.0
2018-03-20, BUY EXECUTED:
Price: 91.51, Cost: -98786.52, Comm: 98.28174000000001
100903.67313999998
2018-03-20, SHORT
order size: -1101.0
2018-03-21, SELL EXECUTED:
Price: 91.39, Cost: -100620.39, Comm: 100.62039
101298.50274999999
2018-03-26, CLOSING
ord

2018-08-30, BUY
order size: 968.0
2018-08-30, Order Canceled/Margin/Rejected
2018-08-30, BUY
order size: 968.0
2018-08-31, Order Canceled/Margin/Rejected
2018-09-04, BUY
order size: 975.0
2018-09-04, Order Canceled/Margin/Rejected
2018-09-04, BUY
order size: 975.0
2018-09-05, Order Canceled/Margin/Rejected
2018-09-05, BUY
order size: 974.0
2018-09-06, BUY EXECUTED:
Price: 107.33, Cost: 104539.42, Comm: 104.53941999999999
107732.17827999993
2018-09-06, CLOSING
order size: -974.0
2018-09-07, SELL EXECUTED:
Price: 107.31, Cost: 104539.42, Comm: 104.51994
107140.65833999994
2018-09-07, BUY
order size: 997.0
2018-09-10, Order Canceled/Margin/Rejected
2018-09-17, BUY
order size: 949.0
2018-09-18, BUY EXECUTED:
Price: 111.23, Cost: 105557.27, Comm: 105.55727000000002
107993.59106999992
2018-09-18, CLOSING
order size: -949.0
2018-09-19, SELL EXECUTED:
Price: 112.09, Cost: 105557.27, Comm: 106.37341
107744.86765999993
2018-09-19, BUY
order size: 960.0
2018-09-20, BUY EXECUTED:
Price: 111.32, Co

order size: 1009.0
2019-02-19, Order Canceled/Margin/Rejected
2019-02-19, BUY
order size: 1009.0
2019-02-20, Order Canceled/Margin/Rejected
2019-02-20, BUY
order size: 1004.0
2019-02-21, BUY EXECUTED:
Price: 106.9, Cost: 107327.6, Comm: 107.3276
110870.35774999998
2019-02-21, CLOSING
order size: -1004.0
2019-02-22, SELL EXECUTED:
Price: 110.05, Cost: 107327.6, Comm: 110.4902
111402.42754999998
2019-02-27, BUY
order size: 996.0
2019-02-27, Order Canceled/Margin/Rejected
2019-02-27, BUY
order size: 996.0
2019-02-28, Order Canceled/Margin/Rejected
2019-02-28, BUY
Final Portfolio Value: 111402.42754999998
